In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from models.detector import face_detector
from models.detector.iris_detector import IrisDetector
from models.verifier.face_verifier import FaceVerifier
import sys
import warnings
warnings.filterwarnings("ignore")
import time
import math

C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\pyth

In [2]:
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-1_keras.h5",  # 2DFAN-4_keras.h5, 2DFAN-2_keras.h5, 2DFAN-1_keras.h5
    fd_type="s3fd"
    # s3fd, mtcnn
)

Instructions for updating:
Colocations handled automatically by placer.
loading S3FD model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, None, None, 6 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 6 0           conv1_2[0][0]                    
______

In [3]:
def resize_image(im, max_size=768):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio), ratio
    else:
        ratio=1
        return im, ratio

recover_size = lambda x: x*(int(1/ratio))

### start build three face databases

In [4]:
start = time.time()
fv = FaceVerifier(classes=512, extractor="insightface")  # extractor="insightface", "facenet"
fv.set_detector(fd)
end = time.time()
print ("cost time: ",end-start)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cost time:  32.152872800827026


In [5]:
name_0, embedding_0 = fv.build_face_identity_database(r".\database\front0",with_detection=True, with_alignment=True)
name_left, embedding_left = fv.build_face_identity_database(r".\database\left45",with_detection=True, with_alignment=False)
name_right, embedding_right = fv.build_face_identity_database(r".\database\right45",with_detection=True, with_alignment=False)

Building FAN for landmarks detection...
loading FAN model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
conv10.23005182903749222_pad (Z (None, 3, 262, 262)  0           input0[0][0]                     
__________________________________________________________________________________________________
conv10.23005182903749222 (Conv2 (None, 64, 128, 128) 9472        conv10.23005182903749222_pad[0][0
__________________________________________________________________________________________________
bn10.20053864458101422 (BatchNo (None, 64, 128, 128) 256         conv10.23005182903749222[0][0]   
_______________________________________

database cost time: 19.996748447418213
database cost time: 0.15670037269592285
Who need to take the photo again:
none
database cost time: 0.12569856643676758
Who need to take the photo again:
none
database cost time: 0.12563109397888184
Who need to take the photo again:
none


In [6]:
def cal_two_points_distance(p1, p2):
    return math.sqrt(math.pow((p2[0] - p1[0]), 2) + math.pow((p2[1] - p1[1]), 2))
def analysis_ratio_to_threshold(ratio, threshold):
    if ratio<2 and ratio>1:
        threshold = 0.6
    if ratio<0.7 and ratio>0.5:
        threshold = 0.6
        
    return threshold

### start detect

In [8]:

threshold = 0.65

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")
    
count = 0
while True:
    start = time.time()
    count= count+1
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    start2 = time.time()
    bboxes, landmarks = fd.detect_face(frame, with_landmarks=True)
    end2 = time.time()
    number = len(bboxes)
    
    if number!=0:
        
        print(f"detect {number} face(s)!")
        print(bboxes)
        
        #開始走訪所有偵測到的臉
        for i in range(number):
            
            #處理臉部 bbox
            x0, y0, x1, y1, score = bboxes[i] 
            x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
            print(x0,y0,x1,y1)
            cv2.rectangle(frame, (y0, x0), (y1, x1), (0, 255, 0), 2)
            crop_face = frame[x0:x1, y0:y1, :]
            size = crop_face.shape
            
            if size[0]!=0 and size[1]!=0:
                
                Xup = x0 - 10 if x0 - 10 > 10 else x0 + 10
                Xdown = x1 - 10 if x1 - 10 > 10 else x1 + 10
                
                #計算距離
                a1,b1 = landmarks[i][2]
                a2,b2 = landmarks[i][30]
                a3,b3 = landmarks[i][14]
                left_dist = round(cal_two_points_distance((b1,a1),(b2,a2)), 2)
                right_dist = round(cal_two_points_distance((b2,a2),(b3,a3)), 2)
                
                if right_dist == 0:
                    right_dist = 0.1
                    
                ratio = left_dist/right_dist
             
                
                
                if b2>=b3 or b2<=b1:
                    
                    cv2.putText(frame, f"angle too large", (y0,Xup), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
                    continue
                
                
                else:
                    
                    if ratio>2:

                        print("right side face, turn left")
                        cv2.putText(frame, f"right face", (y0,Xup), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        dist, min_dist, result = fv.webcam_verify(crop_face, name_right, embedding_right, with_alignment=False,threshold=threshold)

                    elif ratio<0.5:

                        print("left side face, turn right")
                        cv2.putText(frame, f"left face", (y0,Xup), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        dist, min_dist, result = fv.webcam_verify(crop_face, name_left, embedding_left, with_alignment=False,threshold=threshold)


                    else:
                        print("front face")
                        cv2.putText(frame, f"front face", (y0,Xup), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        dist, min_dist, result = fv.webcam_verify(crop_face, name_0, embedding_0, with_alignment=True,threshold=threshold)
                    
          
                
                print("distance:",dist)
                print("min distance:",min_dist)
                print("name:",result)
                
    
                
                label = " {}:{:.2f}".format(result, min_dist)
                cv2.putText(frame, label, (y0, Xdown), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
#                 cv2.putText(frame, f"left_dist:{left_dist}, right_dist:{right_dist}", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
#                 cv2.putText(frame, f"ratio:{ratio}", (10, 60), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            
 
    else:
        
        print("do not detect face(s))!")

    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    end = time.time()
    cost = end-start
    print(f"cccccccccccccccccc:{cost}")
    print("fps:",1/cost)


# release resources
webcam.release()
cv2.destroyAllWindows()    

detect 1 face(s)!
[array([118.06107902526855, 268.46020221710205, 364.3120403289795,
       447.0631380081177, array([0.9999995], dtype=float32)], dtype=object)]
118 268 364 447
front face
distance: [0.5769115527470906, 0.3135061740875244]
min distance: 0.3135061740875244
name: eric
cccccccccccccccccc:0.5766065120697021
fps: 1.7342849570160883
detect 1 face(s)!
[array([120.38586807250977, 269.03512048721313, 365.81100845336914,
       447.64696741104126, array([0.9999995], dtype=float32)],
      dtype=object)]
120 269 365 447
front face
distance: [0.5696797609329224, 0.31035110155741374]
min distance: 0.31035110155741374
name: eric
cccccccccccccccccc:0.1565697193145752
fps: 6.386931038630975
detect 1 face(s)!
[array([120.32533264160156, 268.49793338775635, 364.7565155029297,
       446.3984155654907, array([0.9999995], dtype=float32)], dtype=object)]
120 268 364 446
front face
distance: [0.5775928656260173, 0.3286781152089437]
min distance: 0.3286781152089437
name: eric
ccccccccccccccc

cccccccccccccccccc:0.14192438125610352
fps: 7.046005704935879
detect 1 face(s)!
[array([113.84542846679688, 285.8815059661865, 361.7237854003906,
       467.7053737640381, array([1.], dtype=float32)], dtype=object)]
113 285 361 467
right side face, turn left
distance: [0.2848580201466878]
min distance: 0.2848580201466878
name: eric
cccccccccccccccccc:0.15406250953674316
fps: 6.49087180915682
detect 1 face(s)!
[array([112.28114986419678, 294.7753314971924, 361.016806602478,
       474.8665027618408, array([0.9999995], dtype=float32)], dtype=object)]
112 294 361 474
right side face, turn left
distance: [0.43486167589823405]
min distance: 0.43486167589823405
name: eric
cccccccccccccccccc:0.1552283763885498
fps: 6.442121107399301
detect 1 face(s)!
[array([112.7311954498291, 298.3072814941406, 363.0317783355713,
       478.478271484375, array([0.9999995], dtype=float32)], dtype=object)]
112 298 363 478
right side face, turn left
distance: [0.5074290593465169]
min distance: 0.507429059346516

cccccccccccccccccc:0.1436169147491455
fps: 6.962968127721528
detect 1 face(s)!
[array([121.60064506530762, 247.13743114471436, 367.7844982147217,
       432.32134532928467, array([0.9999995], dtype=float32)],
      dtype=object)]
121 247 367 432
left side face, turn right
distance: [0.5581585168838501]
min distance: 0.5581585168838501
name: eric
cccccccccccccccccc:0.14265847206115723
fps: 7.009748426096301
detect 1 face(s)!
[array([127.83630180358887, 242.31776237487793, 368.32179069519043,
       422.3603801727295, array([0.9999981], dtype=float32)], dtype=object)]
127 242 368 422
left side face, turn right
distance: [0.5961631059646606]
min distance: 0.5961631059646606
name: eric
cccccccccccccccccc:0.14164113998413086
fps: 7.060095676383554
detect 1 face(s)!
[array([132.06788444519043, 241.04975128173828, 368.5613384246826,
       420.3075485229492, array([0.99999714], dtype=float32)],
      dtype=object)]
132 241 368 420
left side face, turn right
distance: [0.6161751826604207]
min 

cccccccccccccccccc:0.15639686584472656
fps: 6.393990024025269
detect 1 face(s)!
[array([118.40722274780273, 264.7024872303009, 367.50451278686523,
       446.1722137928009, array([0.9999995], dtype=float32)], dtype=object)]
118 264 367 446
front face
distance: [0.5419012864430746, 0.29076251983642576]
min distance: 0.29076251983642576
name: eric
cccccccccccccccccc:0.1536695957183838
fps: 6.507468151556854
detect 1 face(s)!
[array([117.10276794433594, 264.2816867828369, 367.17796325683594,
       447.61951637268066, array([0.9999995], dtype=float32)],
      dtype=object)]
117 264 367 447
front face
distance: [0.5564988454182943, 0.2909909725189209]
min distance: 0.2909909725189209
name: eric
cccccccccccccccccc:0.15392255783081055
fps: 6.496773533994839
detect 1 face(s)!
[array([117.20268249511719, 265.27003717422485, 367.54673767089844,
       448.1535668373108, array([0.9999995], dtype=float32)], dtype=object)]
117 265 367 448
front face
distance: [0.5450393040974935, 0.300248082478841

detect 1 face(s)!
[array([109.02388286590576, 269.82438373565674, 366.4125852584839,
       452.6439027786255, array([1.], dtype=float32)], dtype=object)]
109 269 366 452
front face
distance: [0.537254269917806, 0.30067628224690757]
min distance: 0.30067628224690757
name: eric
cccccccccccccccccc:0.14214372634887695
fps: 7.035132859438371
detect 1 face(s)!
[array([113.26044178009033, 269.18406772613525, 364.32159900665283,
       450.4211587905884, array([0.9999995], dtype=float32)], dtype=object)]
113 269 364 450
front face
distance: [0.5414072513580322, 0.3208970069885254]
min distance: 0.3208970069885254
name: eric
cccccccccccccccccc:0.142425537109375
fps: 7.021212770516391
detect 1 face(s)!
[array([112.6380386352539, 270.40048027038574, 364.09944915771484,
       451.01672172546387, array([1.], dtype=float32)], dtype=object)]
112 270 364 451
front face
distance: [0.5411123593648275, 0.2989902178446452]
min distance: 0.2989902178446452
name: eric
cccccccccccccccccc:0.1420066356658935